In [1]:
!dir data/file_3th.json

data/file_3th.json


In [2]:
import os
import json 

f= open(os.path.join('data','file_3th_original.json'), "r", encoding='euc-kr')
read_file= f.read()
f.close()

replace_file= open(os.path.join('data','file_3th.json'), "w", encoding='euc-kr')
for line in read_file.split("\n"):
    replace_file.write(line.replace("\'","\""))
#     replace_file.write(line.replace("\x00",''))
    replace_file.write("\n")
replace_file.close()

# Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
# 위의 에러가 발생하여 모든 "\'"를 "\""로 replace함

In [3]:
with open(os.path.join('data','file_3th.json'), "r", encoding='euc-kr') as line:
    data= json.load(line)

JSONDecodeError: Extra data: line 2 column 1 (char 78934)

Extra data: line 2 column 1 (char 78934) 에러가 발생하여 전체 데이터 load를 진행하지 못했습니다.

해당 에러가 발생하는 이유는 load()는 한 번에 2개 이상의 딕셔너리를 불러올 수 없는데 2개 이상의 딕셔너리를 한 번에 불러오려 하여 생기는 오류로 파악되었습니다.

확인해본 결과 3주차 때 데이터를 불러왔을 때 대용량 데이터를 불러오며 분기별로 나누어진 json(dictionary)가 저장되었고, 그래서 load()가 여러 개의 json을 불러와야 하게 돼 해당 오류가 발생하는 것이었습니다.

검색해본 결과 json file 자체에 손을 대서 수정해야 하는 것 같은데, 그러기엔 데이터가 너무 방대하여 수정할 수 없다고 판단하였습니다.(다른 해결방법이 있을지도 모르나, 일단 아무리 찾아봐도 발견할 수 없었습니다...)

그렇기에 결국 전체 데이터를 load()하는 것은 어려울 것 같아 포기하였고, loads()로 하나의 json만 불러와 진행하는 것으로 하겠습니다... 죄송합니다.

In [6]:
f= open(os.path.join('data','file_3th.json'), "r", encoding='euc-kr')
for line in f:
    data= json.loads(line)
    break

### 9-1: 문제 설정

3주차 과제에 수집한 데이터: 서울시 우리마을가게 상권분석서비스(상권-직장인구)


설정한 문제1: 총직장인구수가 가장 많은 지역은?

설정한 문제2: 해당 지역은 남성직장인구수가 더 많은가, 여성직장인구수가 더 많은가?

### 9-2: 문제의 답

In [7]:
import pyspark
import os

myConf= pyspark.SparkConf()

spark= pyspark.sql.SparkSession\
.builder\
.master("local")\
.appName("myAPP")\
.config(conf=myConf)\
.getOrCreate()

In [8]:
myrdd= spark.sparkContext.parallelize(data['VwsmTrdarWrcPopltnQq']['row'])

In [9]:
myrdd.collect()

[{'STDR_YY_CD': '2020',
  'STDR_QU_CD': '1',
  'TRDAR_SE_CD': 'A',
  'TRDAR_SE_CD_NM': '골목상권',
  'TRDAR_CD': '1000001',
  'TRDAR_CD_NM': '계동길',
  'TOT_WRC_POPLTN_CO': 842.0,
  'ML_WRC_POPLTN_CO': 388.0,
  'FML_WRC_POPLTN_CO': 454.0,
  'AGRDE_10_WRC_POPLTN_CO': 0.0,
  'AGRDE_20_WRC_POPLTN_CO': 97.0,
  'AGRDE_30_WRC_POPLTN_CO': 160.0,
  'AGRDE_40_WRC_POPLTN_CO': 190.0,
  'AGRDE_50_WRC_POPLTN_CO': 191.0,
  'AGRDE_60_ABOVE_WRC_POPLTN_CO': 204.0,
  'MAG_10_WRC_POPLTN_CO': 0.0,
  'MAG_20_WRC_POPLTN_CO': 30.0,
  'MAG_30_WRC_POPLTN_CO': 55.0,
  'MAG_40_WRC_POPLTN_CO': 85.0,
  'MAG_50_WRC_POPLTN_CO': 75.0,
  'MAG_60_ABOVE_WRC_POPLTN_CO': 143.0,
  'FAG_10_WRC_POPLTN_CO': 0.0,
  'FAG_20_WRC_POPLTN_CO': 67.0,
  'FAG_30_WRC_POPLTN_CO': 105.0,
  'FAG_40_WRC_POPLTN_CO': 105.0,
  'FAG_50_WRC_POPLTN_CO': 116.0,
  'FAG_60_ABOVE_WRC_POPLTN_CO': 61.0},
 {'STDR_YY_CD': '2020',
  'STDR_QU_CD': '1',
  'TRDAR_SE_CD': 'A',
  'TRDAR_SE_CD_NM': '골목상권',
  'TRDAR_CD': '1000002',
  'TRDAR_CD_NM': '난계로27길',
  'TOT_W

In [10]:
myDf= spark.createDataFrame(myrdd)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pyspark/sql/session.py:366: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [11]:
myDf.printSchema()

root
 |-- AGRDE_10_WRC_POPLTN_CO: double (nullable = true)
 |-- AGRDE_20_WRC_POPLTN_CO: double (nullable = true)
 |-- AGRDE_30_WRC_POPLTN_CO: double (nullable = true)
 |-- AGRDE_40_WRC_POPLTN_CO: double (nullable = true)
 |-- AGRDE_50_WRC_POPLTN_CO: double (nullable = true)
 |-- AGRDE_60_ABOVE_WRC_POPLTN_CO: double (nullable = true)
 |-- FAG_10_WRC_POPLTN_CO: double (nullable = true)
 |-- FAG_20_WRC_POPLTN_CO: double (nullable = true)
 |-- FAG_30_WRC_POPLTN_CO: double (nullable = true)
 |-- FAG_40_WRC_POPLTN_CO: double (nullable = true)
 |-- FAG_50_WRC_POPLTN_CO: double (nullable = true)
 |-- FAG_60_ABOVE_WRC_POPLTN_CO: double (nullable = true)
 |-- FML_WRC_POPLTN_CO: double (nullable = true)
 |-- MAG_10_WRC_POPLTN_CO: double (nullable = true)
 |-- MAG_20_WRC_POPLTN_CO: double (nullable = true)
 |-- MAG_30_WRC_POPLTN_CO: double (nullable = true)
 |-- MAG_40_WRC_POPLTN_CO: double (nullable = true)
 |-- MAG_50_WRC_POPLTN_CO: double (nullable = true)
 |-- MAG_60_ABOVE_WRC_POPLTN_CO: doubl

In [12]:
myDf.show()

+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------------+----------------+----------+----------+-----------------+--------+--------------+-----------+--------------+
|AGRDE_10_WRC_POPLTN_CO|AGRDE_20_WRC_POPLTN_CO|AGRDE_30_WRC_POPLTN_CO|AGRDE_40_WRC_POPLTN_CO|AGRDE_50_WRC_POPLTN_CO|AGRDE_60_ABOVE_WRC_POPLTN_CO|FAG_10_WRC_POPLTN_CO|FAG_20_WRC_POPLTN_CO|FAG_30_WRC_POPLTN_CO|FAG_40_WRC_POPLTN_CO|FAG_50_WRC_POPLTN_CO|FAG_60_ABOVE_WRC_POPLTN_CO|FML_WRC_POPLTN_CO|MAG_10_WRC_POPLTN_CO|MAG_20_WRC_POPLTN_CO|MAG_30_WRC_POPLTN_CO|MAG_40_WRC_POPLTN_CO|MAG_50_WRC_POPLTN_CO|MAG_60_ABOVE_WRC_POPLTN_CO|ML_WRC_POPLTN_CO|STDR_QU_CD|STDR_YY_C

#### 문제1: 총직장인구수가 가장 많은 지역은?

In [13]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

TOT_max= Window.partitionBy().orderBy(F.col("TOT_WRC_POPLTN_CO").desc())

In [14]:
from pyspark.sql.functions import rank

tot_df= myDf.withColumn("rank", rank().over(TOT_max))

In [15]:
tot_df.select(tot_df["TRDAR_CD_NM"], tot_df["TOT_WRC_POPLTN_CO"], tot_df["rank"]).show()

+--------------+-----------------+----+
|   TRDAR_CD_NM|TOT_WRC_POPLTN_CO|rank|
+--------------+-----------------+----+
|  한강대로52길|           8713.0|   1|
| 한강대로104길|           8152.0|   2|
|        명동길|           7830.0|   3|
|        평창길|           6654.0|   4|
|   장충단로8길|           3778.0|   5|
|돈화문로11가길|           3686.0|   6|
|     삼청로5길|           2703.0|   7|
|    동호로20길|           2429.0|   8|
|     뚝섬로5길|           2136.0|   9|
|  우사단로14길|           1987.0|  10|
|     무학로6길|           1897.0|  11|
|  한강대로43길|           1841.0|  12|
|      백석동길|           1797.0|  13|
|   삼일대로4길|           1571.0|  14|
|    동호로10길|           1553.0|  15|
|    원효로89길|           1387.0|  16|
|    청파로71길|           1317.0|  17|
|    동호로15길|           1161.0|  18|
| 두텁바위로1길|           1111.0|  19|
|    퇴계로46길|           1098.0|  20|
+--------------+-----------------+----+
only showing top 20 rows



분석 결과 '한강대로52길'이 총직장인구수 8713명으로, 서울시 내에서 가장 총직장인구수가 많은 것으로 파악되었다.

#### 문제2: 지역별로 남성직장인구수가 더 많은가, 여성직장인구수가 더 많은가?

In [16]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import FloatType

mf_udf= udf(lambda female, male: " M (M > F)" if (female <= male) else " F (M < F)", StringType())
mf_udf2= udf(lambda female, male: (male-female) if (female <= male) else (female-male), FloatType())

MF_df= myDf.select(myDf["TRDAR_CD_NM"], myDf["ML_WRC_POPLTN_CO"], myDf["FML_WRC_POPLTN_CO"])\
    .withColumn("F/M", mf_udf(myDf["FML_WRC_POPLTN_CO"], myDf["ML_WRC_POPLTN_CO"]))\
    .withColumn("  F/M SUB", mf_udf2(myDf["FML_WRC_POPLTN_CO"], myDf["ML_WRC_POPLTN_CO"]))
MF_df.show()

+--------------+----------------+-----------------+----------+---------+
|   TRDAR_CD_NM|ML_WRC_POPLTN_CO|FML_WRC_POPLTN_CO|       F/M|  F/M SUB|
+--------------+----------------+-----------------+----------+---------+
|        계동길|           388.0|            454.0| F (M < F)|     66.0|
|    난계로27길|           114.0|             87.0| M (M > F)|     27.0|
|돈화문로11가길|          2700.0|            986.0| M (M > F)|   1714.0|
|        명륜길|            32.0|             24.0| M (M > F)|      8.0|
|      백석동길|          1066.0|            731.0| M (M > F)|    335.0|
|    북촌로11길|           232.0|            439.0| F (M < F)|    207.0|
|     북촌로5길|           419.0|             84.0| M (M > F)|    335.0|
|   북촌로5나길|           113.0|            121.0| F (M < F)|      8.0|
|     삼청로5길|          2626.0|             77.0| M (M > F)|   2549.0|
|   성균관로5길|            84.0|             64.0| M (M > F)|     20.0|
|      숭인동길|           133.0|            196.0| F (M < F)|     63.0|
|        옥인길|           

"F/M" column을 통해 각 지역별로 남성직장인구수가 더 많은지, 여성직장인구수가 더 많은지 파악할 수 있고, 'F/M SUB' column을 통해 얼마나 더 많은지 파악할 수 있다.